In [25]:
import ffmpeg
# config
import common.init as init
from singleton_pattern.load_model import get_model
from  singleton_pattern.load_config import get_config
from singleton_pattern.load_dataset_reader import get_test_reader
from singleton_pattern.load_data_generator import create_tensor_data_generator
from common.cache import Cache
from common.cuda_info import get_device
from singleton_pattern.load_config import get_non_dnn_method_list
from common.cache import CacheType
from singleton_pattern.load_model import get_model
from loss.pearson import PearsonLoss
import shutil
import os
import numpy as np
import cv2
import concurrent.futures  
import traceback
import time
import threading
import pandas as pd
import matplotlib.pyplot as plt
# config_name = 'PhysNet.yaml'
config_name = 'POS.yaml'
# config_name = 'PCA.yaml'


In [26]:
init.run(config_name,False)

root_output_path = './cache/COMPRESS'
os.makedirs(root_output_path, exist_ok=True)
config = get_config()
method = config.get('method')
config['test']['dataset']['force_clear_cache'] = True
ploss = PearsonLoss()
# 创建互斥锁
save_psnr_to_sheet_lock = threading.Lock()
save_ssim_to_sheet_lock = threading.Lock()
save_compression_ratio_to_sheet_lock = threading.Lock()
save_pearson_and_snr_lock = threading.Lock()
drop_lock = threading.Lock()
test_dataloader_lock = threading.Lock()

def calculate(codec,suffix, mode,compression_strength):
    name = f'{codec}_{mode}_{str(compression_strength)}'
    # load test dataset
    test_reader = get_test_reader()
    video_paths,ppgs = test_reader.load_data(print_info = False)
    codec_video_paths = [os.path.abspath(f'./cache/COMPRESS/{codec}/{mode}_{str(compression_strength)}_{str(i)}.{suffix}') for i in  range(len(video_paths))]
    '''
        compress
    '''
    compress(video_paths,codec_video_paths,codec,mode, compression_strength)
    '''
        compression_ratios
    '''
    # calculate_compression_ratio(video_paths,codec_video_paths,name)
    '''
        video psnr, ssim
    '''
    # calculate_video_metrics(video_paths,codec_video_paths,codec,mode,compression_strength)

    '''
        pearson and snr
    '''
    # calculate_pearson_and_snr(codec_video_paths,ppgs,name)



def compress(video_paths,output_video_paths,codec,mode, compression_strength):
    for i in range(len(video_paths)):
        video_path = video_paths[i]
        output_video_path = output_video_paths[i]
        bitrate = get_video_bitrate(video_path)
        dir = os.path.dirname(output_video_path)
        os.makedirs(dir, exist_ok=True)
        param = {
            'codec':codec,
            'pix_fmt':'yuv422p',
            'gpu':'auto'
        }
        if mode == 'b':
            compression_strength = int(bitrate - compression_strength * (bitrate-100000)/51)
            param['maxrate'] = (bitrate + compression_strength)/2
        param[mode] = compression_strength
        if os.path.exists(output_video_path):
            os.remove(output_video_path)
        steam = ffmpeg.input(video_path)
        steam = steam.output(output_video_path, **param,)
        steam.run()

def calculate_compression_ratio(video_paths, codec_video_paths,sheet_name):
    compression_ratios = []
    video_bitrates = []
    for index in range(len(video_paths)):
        video_path = video_paths[index]
        codec_video_path = codec_video_paths[index]
        original_size = os.path.getsize(video_path)
        codec_size = os.path.getsize(codec_video_path)
        ratio = codec_size / original_size
        compression_ratios.append([ratio])
        video_bitrates.append([get_video_bitrate(codec_video_path)])
    compression_ratios = np.array(compression_ratios).T
    video_bitrates = np.array(video_bitrates).T
    # 保存
    with save_compression_ratio_to_sheet_lock:
        save_to_sheet('compression_ratios',sheet_name,compression_ratios)
        save_to_sheet('video_bitrates',sheet_name,video_bitrates)

    
def calculate_video_metrics(video_paths,codec_video_paths,codec,mode,intensity):
    videos_psnrs = []
    videos_ssims = []
    for index in range(len(video_paths)):
        video_path = video_paths[index]
        codec_video_path = codec_video_paths[index]
        # break
        psnrs = []
        ssims = []
        cap1 = cv2.VideoCapture(video_path)
        cap2 = cv2.VideoCapture(codec_video_path)
        index = 0
        sleep_time = 0
        while True:
            ret1, frame_1 = cap1.read()
            ret2, frame_2 = cap2.read()
            if not ret1 or not ret2:
                if sleep_time < 120 and index == 0:
                    sleep_time += 1
                    time.sleep(1)
                    continue
                break
            if index > 200 and index < 220:
                save_frame_diff(frame_1,frame_2,f'out/img/diff/{codec}/{mode}__{str(intensity)}_{str(index)}.png')
            psnr = PSNR(frame_1,frame_2)
            psnrs.append(psnr)
            # ssim = SSIM(frame_1, frame_2)
            # ssims.append(ssim)
            index += 1
        videos_psnrs.append(psnrs)
        videos_ssims.append(ssims)
        cap1.release()
        cap2.release()
        if sleep_time >= 120:
            raise Exception(f'无法加载视频：{codec_video_paths[index]}')
    videos_psnrs = np.array(videos_psnrs).T
    videos_ssims = np.array(videos_ssims).T
    # 保存
    sheet_name = f'{codec}_{mode}_{str(intensity)}'
    with save_psnr_to_sheet_lock:
        save_to_sheet('psnr',sheet_name,videos_psnrs)
    with save_ssim_to_sheet_lock:
        save_to_sheet('ssim',sheet_name,videos_ssims)
    pass

def calculate_pearson_and_snr(codec_video_paths,ppgs,name):
    test_data_generator = create_tensor_data_generator(CacheType.TEST_SYNC)
    test_dataloader = test_data_generator.get_tensor_dataloader((codec_video_paths,ppgs),print_info = False)
    non_dnn_method_list = get_non_dnn_method_list()
    is_need_train = config['method'] not in non_dnn_method_list
    model = get_model()
    if is_need_train:
        cache_model = Cache(CacheType.MODEL).read_model()
        model.load_state_dict(cache_model.state_dict())
    model.eval()
    gpu_device = get_device()
    model.to(gpu_device)

    pearsons = []
    snrs = []
    b_num = 0
    with test_dataloader_lock:
        for batch_X, batch_y in test_dataloader:
            batch_X = batch_X.to(gpu_device)
            batch_y = batch_y.cpu()
            outputs = model.forward(batch_X).cpu()
            p = ploss.forward(batch_y,outputs)
            s = SNR(batch_y,outputs)
            pearsons.append(p)
            snrs.append(s)
            bvp_image_path = f'out/img/bvp/{config["method"]}/{name}'
            os.makedirs(bvp_image_path, exist_ok=True)
            with drop_lock:
                drop(f'{bvp_image_path}/{str(b_num)}.png',batch_y,outputs)
            b_num += 1
    datas = {
        'pearson':np.array(pearsons),
        'snr':np.array(snrs)
    }
    with save_pearson_and_snr_lock:
        save_pearson_and_snr(name,datas)


def save_frame_diff(frame_1,frame_2,file_path):
    subtracted_frame = cv2.absdiff(frame_1, frame_2)
    gray_frame = cv2.cvtColor(subtracted_frame,cv2.COLOR_BGR2GRAY)
    gamma = 0.3
    gamma_corrected_frame = np.power(gray_frame / 255.0, gamma) * 255.0
    gamma_corrected_frame = np.uint8(gamma_corrected_frame)
    directory = os.path.dirname(file_path)
    os.makedirs(directory, exist_ok=True)
    cv2.imwrite(file_path,cv2.cvtColor(gamma_corrected_frame,cv2.COLOR_GRAY2BGR))

def save_to_sheet(name,sheet_name,datas):
    file_path = f'out/{name}.xlsx'
    if not os.path.exists(file_path):
        pd.DataFrame().to_excel(file_path)
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
        try:
            writer.book.remove(writer.book[sheet_name])
        except:
            pass
        df = pd.DataFrame(datas)
        df.to_excel(writer, index=False, sheet_name=sheet_name,header=[f'video_{str(i+1)}' for i in range(datas.shape[1])])



def save_pearson_and_snr(sheet_name,datas):
    file_path = f'out/{config["method"]}_pearson_and_snr.xlsx'
    if not os.path.exists(file_path):
        pd.DataFrame().to_excel(file_path)
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl') as writer:
        try:
            writer.book.remove(writer.book[sheet_name])
        except:
            pass
        df = pd.DataFrame(datas)
        df.to_excel(writer, index=False, sheet_name=sheet_name,header=True)

def drop(file_path,y_1,y_2):
    y_1 = y_1.cpu().numpy().flatten()
    y_2 = y_2.cpu().numpy().flatten()
    plt.clf()
    plt.plot(y_1)
    plt.plot(y_2)
    plt.savefig(file_path)
    # plt.show()

def get_video_bitrate(video_path):
    # 使用 ffmpeg 获取视频元数据
    probe = ffmpeg.probe(video_path)

    # 获取视频流信息
    video_stream = next(s for s in probe['streams'] if s['codec_type'] == 'video')

    # 提取比特率信息
    bitrate = video_stream['bit_rate']
    return int(bitrate)

def PSNR(frame_1,frame_2):
    f_pow = np.power(frame_1 - frame_2,2)
    average_per_channel = np.mean(f_pow, axis=-1)
    mse = average_per_channel/(frame_1.shape[0] * frame_1.shape[1])
    if (np.sum(mse)/3) == 0:
        return np.inf
    return 10 * np.log10(np.power(255,2) / (np.sum(mse)/3))

def SSIM(frame_1, frame_2, K1=0.01, K2=0.03, L=255):
    C1 = (K1 * L) ** 2
    C2 = (K2 * L) ** 2
    mean_1 = np.mean(frame_1, axis=(0, 1))
    mean_2 = np.mean(frame_2, axis=(0, 1))
    var_1 = np.var(frame_1, axis=(0, 1))
    var_2 = np.var(frame_2, axis=(0, 1))
    covariance = np.cov(np.ravel(frame_1), np.ravel(frame_2))
    numerator = (2 * mean_1 * mean_2 + C1) * (2 * covariance + C2)
    denominator = (mean_1 ** 2 + mean_2 ** 2 + C1) * (var_1 + var_2 + C2)
    return np.mean(numerator / denominator)

def SNR(y_1,y_2):
    y_1 = y_1.cpu().numpy().flatten()
    y_2 = y_2.cpu().numpy().flatten()
    # 计算信号和噪声的功率
    signal_power = np.mean(y_1 ** 2)
    noise_power = np.mean((y_1 - y_2) ** 2)
    # 检查噪声功率是否为0
    if noise_power == 0:
        return float('inf')  # 返回正无穷，表示信号远远大于噪声
    # 计算信噪比
    snr = 10 * np.log10(signal_power / noise_power)
    
    return snr

def task_wrap(*args, **kwargs):
    try:
        calculate(*args, **kwargs)
        pass
    except Exception  as e:
        print(e)
        traceback.print_exc()
    print(*args, str(kwargs))
    # print(str(threading.get_ident()))

In [27]:
# mode = 'b'
# mode = 'qp'
mode = 'crf'
# codes = [('hevc','mp4')]
# codes = [('h264','avi')]
# codes = [('h264','avi'),('hevc','mp4')]
# codes = [('libxvid','avi')]
codes = [('libaom-av1','mkv')]
# compression_strengths = range(0, 36)
compression_strengths = range(0, 2)

In [28]:
for codec,suffix in codes:
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        tasks = []
        for compression_strength in compression_strengths:
            param = {'codec':codec,'suffix':suffix,'mode':mode,'compression_strength':compression_strength}
            tasks.append(executor.submit(task_wrap, **param))
        for future in concurrent.futures.as_completed(tasks):
            pass

ffmpeg error (see stderr output for detail)
{'codec': 'libaom-av1', 'suffix': 'mkv', 'mode': 'crf', 'compression_strength': 3}


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 259, in task_wrap
    calculate(*args, **kwargs)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 26, in calculate
    compress(video_paths,codec_video_paths,codec,mode, compression_strength)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 63, in compress
    steam.run()
  File "c:\ProgramData\miniconda3\envs\compression\lib\site-packages\ffmpeg\_run.py", line 325, in run
    raise Error('ffmpeg', out, err)
ffmpeg._run.Error: ffmpeg error (see stderr output for detail)


ffmpeg error (see stderr output for detail)
{'codec': 'libaom-av1', 'suffix': 'mkv', 'mode': 'crf', 'compression_strength': 4}


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 259, in task_wrap
    calculate(*args, **kwargs)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 26, in calculate
    compress(video_paths,codec_video_paths,codec,mode, compression_strength)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 63, in compress
    steam.run()
  File "c:\ProgramData\miniconda3\envs\compression\lib\site-packages\ffmpeg\_run.py", line 325, in run
    raise Error('ffmpeg', out, err)
ffmpeg._run.Error: ffmpeg error (see stderr output for detail)


ffmpeg error (see stderr output for detail)
{'codec': 'libaom-av1', 'suffix': 'mkv', 'mode': 'crf', 'compression_strength': 5}


...

Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 259, in task_wrap
    calculate(*args, **kwargs)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 26, in calculate
    compress(video_paths,codec_video_paths,codec,mode, compression_strength)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 63, in compress
    steam.run()
  File "c:\ProgramData\miniconda3\envs\compression\lib\site-packages\ffmpeg\_run.py", line 325, in run
    raise Error('ffmpeg', out, err)
ffmpeg._run.Error: ffmpeg error (see stderr output for detail)


ffmpeg error (see stderr output for detail)
{'codec': 'libaom-av1', 'suffix': 'mkv', 'mode': 'crf', 'compression_strength': 18}


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 259, in task_wrap
    calculate(*args, **kwargs)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 26, in calculate
    compress(video_paths,codec_video_paths,codec,mode, compression_strength)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 63, in compress
    steam.run()
  File "c:\ProgramData\miniconda3\envs\compression\lib\site-packages\ffmpeg\_run.py", line 325, in run
    raise Error('ffmpeg', out, err)
ffmpeg._run.Error: ffmpeg error (see stderr output for detail)


ffmpeg error (see stderr output for detail)
{'codec': 'libaom-av1', 'suffix': 'mkv', 'mode': 'crf', 'compression_strength': 1}


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 259, in task_wrap
    calculate(*args, **kwargs)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 26, in calculate
    compress(video_paths,codec_video_paths,codec,mode, compression_strength)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_19636\3132404948.py", line 63, in compress
    steam.run()
  File "c:\ProgramData\miniconda3\envs\compression\lib\site-packages\ffmpeg\_run.py", line 325, in run
    raise Error('ffmpeg', out, err)
ffmpeg._run.Error: ffmpeg error (see stderr output for detail)


In [ ]:
# for codec,suffix in codes:
#     for crf in crfs:
#         calculate(codec,suffix,crf=crf)


In [ ]:
# shutil.rmtree(root_output_path,ignore_errors=True)